In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
print("created spark session.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-02-03 18:29:45,474 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2023-02-03 18:29:45,474 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2023-02-03 18:29:45,474 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
